In [1]:
import mbuild as mb
import os

/home/pranay/miniconda3/envs/mbuild/lib/python3.11/site-packages/lark/utils.py:163: DeprecationWarning: module 'sre_parse' is deprecated
  import sre_parse
/home/pranay/miniconda3/envs/mbuild/lib/python3.11/site-packages/lark/utils.py:164: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants
/home/pranay/mbuild/mbuild/recipes/__init__.py:13: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entry_points = metadata.entry_points()["mbuild.plugins"]


In [2]:
# Setting up basic classes for storage

num_molecules = 200
atoms_per_mol = 324


if(os.path.exists("system.gsd")):
    os.remove("system.gsd")
class Atom:
    def __init__(self, n, mol, typeid, charge, x, y, z):
        self.n = n
        self.mol = mol
        self.charge = charge
        self.x = x
        self.y = y
        self.z = z
        
        
        # Mapping types is based on GAFF; Todo: make a separate file for mapping atoms.

        if(typeid <= 18):
            self.atomtype = "C"

        elif(typeid > 18 and typeid <= 31):
            self.atomtype = "H"

        elif(typeid == 32):
            self.atomtype = "F"
        
        elif(typeid == 33):
            self.atomtype = "Cl"

        elif(typeid == 34):
            self.atomtype = "Br"

        elif(typeid == 35):
            self.atomtype = "I"

        elif(typeid >= 36 and typeid <= 48):
            self.atomtype = "N"

        elif(typeid >= 49 and typeid <= 52):
            self.atomtype = "O"

        elif (typeid >= 53 and typeid <= 63):
            self.atomtype = "P"

        elif (typeid >= 64 and typeid <= 71):
            self.atomtype = "S"

        else:
            self.atomtype = " "

class Bond:
    def __init__(self, a1, a2):
        self.atom1 = a1
        self.atom2 = a2

In [3]:
# File input

fc = open("sorted_coords.data", "r")
fb = open("sorted_bonds.data", "r")

clines = fc.readlines()
blines = fb.readlines()

atoms = []

for line in clines:
    tokens = line.split()
    #print(tokens)    
    n = int(tokens[0])
    mol = int(tokens[1])
    typeid = int(tokens[2])
    charge = float(tokens[3])
    x = float(tokens[4])
    y = float(tokens[5])
    z = float(tokens[6])
    
    atoms.append(Atom(n, mol, typeid, charge, x, y, z))

bondi = []
bondj = []

for line in blines:
    tokens = line.split()
    #print(tokens)
    atom1 = int(tokens[2])
    atom2 = int(tokens[3])

    bondi.append(atom1)
    bondj.append(atom2)

system = mb.Compound()

In [4]:
%%time

# Timing particle additions

for i in range(num_molecules):
    m = i + 1
    mol = mb.Compound()

    for atom in atoms:
        if atom.mol == m:
            a = mb.Particle(pos=[atom.x, atom.y, atom.z], element = atom.atomtype, name=atom.atomtype, charge= atom.charge)
            mol.add(a)

    system.add(mol)

CPU times: user 39.3 s, sys: 524 ms, total: 39.8 s
Wall time: 40.1 s


In [5]:
print(f"Added {system.n_particles} atoms")

Added 64800 atoms


In [6]:
%%time

# Timing bond additions

for i in range(len(bondi)):
    #print(f"Adding bond number {i}")
    system.add_bond((system[bondi[i] - 1], system[bondj[i] - 1]))

CPU times: user 2h 42min 50s, sys: 1.04 s, total: 2h 42min 51s
Wall time: 2h 43min 23s


In [7]:
print(f"Added {system.n_bonds} bonds")

Added 78400 bonds
